## 確認套件已經安裝 & Get data

In [1]:
from __future__ import print_function
!pip install -q keras
import keras
from google.colab import files
import numpy as np
from past.builtins import execfile
import numpy as np
import os

from keras.utils import np_utils
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

Using TensorFlow backend.


## 請將本機端的資料 "pkgo_city66_cass5_v1.csv" 上傳到 colab 環境

In [0]:
if not os.path.exists("pkgo_city66_class5_v1.csv"):
  uploaded = files.upload()

def readInput(file):
  ''' Read input files '''
  my_data = np.genfromtxt(file, delimiter=',',skip_header=1)

  ''' The first column to the 199th column is used as input features '''
  X_train = my_data[:,0:200]
  X_train = X_train.astype('float32')

  ''' The 200-th column is the answer '''
  y_train = my_data[:,200]
  y_train = y_train.astype('int')

  ''' Convert to one-hot encoding '''
  Y_train = np_utils.to_categorical(y_train,5)

  ''' Shuffle training data '''
  X_train,Y_train = shuffle(X_train,Y_train,random_state=100)
  return X_train, Y_train

X_train, Y_train = readInput(file="pkgo_city66_class5_v1.csv")

## 開始建立第一個模型 
### (資料已經先替各位做過簡單的預處理囉)

In [0]:
''' Import keras to build a DL model '''
from keras.models import Model
from keras.layers import Dense, Activation, Input
from keras.optimizers import SGD, Adam, RMSprop, Adagrad
from keras.regularizers import l1, l2, l1_l2
import keras.backend as K

In [0]:
#@title How BN and batch size affect performance { vertical-output: true, display-mode: "both" }
from keras.layers import BatchNormalization
batch_size_list = [1, 2, 4, 8, 16, 32, 64] #@param {type:"raw"}

from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor = 'val_loss', 
                               patience = 3, 
                               verbose = 1)

train_loss_list, train_acc_list = [], []
valid_loss_list, valid_acc_list = [], []

for batch_size in batch_size_list:
  print('Building a model with batch_size: %g' % (batch_size))
  
  K.clear_session()
  x_input = Input(shape = [X_train.shape[1]])

  x = Dense(128)(x_input)
  x = BatchNormalization(axis = -1)(x)
  x = Activation('relu')(x)
  

  x = Dense(256)(x)
  x = BatchNormalization(axis = -1)(x)
  x = Activation('relu')(x)
  
  x = Dense(5)(x)
  out = Activation('softmax')(x)

  model = Model(inputs=[x_input], outputs=[out])

  optim = Adam(lr = 0.001)

  model.compile(loss='categorical_crossentropy',
          optimizer=optim,
          metrics=['accuracy'])
  #model.summary()

  epochs = 50

  history = model.fit(X_train, Y_train,
                batch_size=batch_size,
                epochs=epochs,
                verbose=1,
                shuffle=True,
                validation_split=0.1,
                #callbacks = [early_stopping]
                     )

  loss	= history.history.get('loss')
  acc 	= history.history.get('acc')

  val_loss = history.history.get('val_loss')
  val_acc = history.history.get('val_acc')
  
  train_loss_list.append(loss)
  valid_loss_list.append(val_loss)
  train_acc_list.append(acc)
  valid_acc_list.append(val_acc)

  
list_len = len(train_loss_list)

# training loss
for i in np.arange(list_len):
  plt.plot(np.arange(len(train_loss_list[i])), train_loss_list[i], label = 'batch_size:%g' % (batch_size_list[i]) )

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

# training acc
for i in np.arange(list_len):
  plt.plot(np.arange(len(train_acc_list[i])), train_acc_list[i], label = 'batch_size:%g' % (batch_size_list[i]) )

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

# valid loss
for i in np.arange(list_len):
  plt.plot(np.arange(len(valid_loss_list[i])), valid_loss_list[i], label = 'batch_size:%g' % (batch_size_list[i]))

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()


# valid acc
for i in np.arange(list_len):
  plt.plot(np.arange(len(valid_acc_list[i])), valid_acc_list[i], label = 'batch_size:%g' % (batch_size_list[i]))

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()



Building a model with batch_size: 1
Train on 1652 samples, validate on 184 samples
Epoch 1/50
1652/1652 [==============================] - 5s 3ms/step - loss: 1.4730 - acc: 0.5133 - val_loss: 13.7529 - val_acc: 0.1467
Epoch 2/50
1652/1652 [==============================] - 5s 3ms/step - loss: 1.3692 - acc: 0.5163 - val_loss: 13.9282 - val_acc: 0.1359
Epoch 3/50
1652/1652 [==============================] - 5s 3ms/step - loss: 1.3549 - acc: 0.5163 - val_loss: 13.6832 - val_acc: 0.1467
Epoch 4/50
 517/1652 [========>.....................] - ETA: 3s - loss: 1.3712 - acc: 0.5029

1652/1652 [==============================] - 5s 3ms/step - loss: 1.3529 - acc: 0.5163 - val_loss: 13.8124 - val_acc: 0.1413
Epoch 5/50
1652/1652 [==============================] - 5s 3ms/step - loss: 1.3527 - acc: 0.5163 - val_loss: 13.4927 - val_acc: 0.1630
Epoch 6/50
1652/1652 [==============================] - 5s 3ms/step - loss: 1.3526 - acc: 0.5163 - val_loss: 13.5777 - val_acc: 0.1576
Epoch 7/50
1652/1652 [==============================] - 5s 3ms/step - loss: 1.3526 - acc: 0.5163 - val_loss: 13.7529 - val_acc: 0.1467
Epoch 8/50
  37/1652 [..............................] - ETA: 4s - loss: 1.5638 - acc: 0.3784

1652/1652 [==============================] - 5s 3ms/step - loss: 1.3525 - acc: 0.5163 - val_loss: 13.5797 - val_acc: 0.1576
Epoch 9/50
1652/1652 [==============================] - 5s 3ms/step - loss: 1.3525 - acc: 0.5163 - val_loss: 13.9571 - val_acc: 0.1304
Epoch 10/50
1652/1652 [==============================] - 5s 3ms/step - loss: 1.3526 - acc: 0.5163 - val_loss: 13.7529 - val_acc: 0.1467
Epoch 11/50
1642/1652 [============================>.] - ETA: 0s - loss: 1.3520 - acc: 0.5164

1652/1652 [==============================] - 5s 3ms/step - loss: 1.3526 - acc: 0.5163 - val_loss: 13.5932 - val_acc: 0.1522
Epoch 12/50
1652/1652 [==============================] - 5s 3ms/step - loss: 1.3524 - acc: 0.5163 - val_loss: 13.7551 - val_acc: 0.1467
Epoch 13/50
1652/1652 [==============================] - 5s 3ms/step - loss: 1.3525 - acc: 0.5163 - val_loss: 13.5209 - val_acc: 0.1576
Epoch 14/50
1652/1652 [==============================] - 5s 3ms/step - loss: 1.3524 - acc: 0.5163 - val_loss: 13.5777 - val_acc: 0.1576
Epoch 15/50
 135/1652 [=>............................] - ETA: 4s - loss: 1.4448 - acc: 0.4667

1652/1652 [==============================] - 5s 3ms/step - loss: 1.3525 - acc: 0.5163 - val_loss: 13.8405 - val_acc: 0.1413
Epoch 16/50
1652/1652 [==============================] - 5s 3ms/step - loss: 1.3526 - acc: 0.5163 - val_loss: 13.5289 - val_acc: 0.1576
Epoch 17/50
1652/1652 [==============================] - 5s 3ms/step - loss: 1.3526 - acc: 0.5163 - val_loss: 13.4901 - val_acc: 0.1630
Epoch 18/50
1609/1652 [============================>.] - ETA: 0s - loss: 1.3506 - acc: 0.5183

...

1652/1652 [==============================] - 0s 143us/step - loss: 0.7343 - acc: 0.7421 - val_loss: 0.9039 - val_acc: 0.6902
Epoch 9/50
1652/1652 [==============================] - 0s 134us/step - loss: 0.7151 - acc: 0.7433 - val_loss: 0.9978 - val_acc: 0.6522
Epoch 10/50
1652/1652 [==============================] - 0s 133us/step - loss: 0.6670 - acc: 0.7730 - val_loss: 0.8033 - val_acc: 0.6957
Epoch 11/50
1652/1652 [==============================] - 0s 135us/step - loss: 0.6577 - acc: 0.7609 - val_loss: 1.0728 - val_acc: 0.5707
Epoch 12/50
1652/1652 [==============================] - 0s 136us/step - loss: 0.6183 - acc: 0.7827 - val_loss: 1.0409 - val_acc: 0.6413
Epoch 13/50
1652/1652 [==============================] - 0s 132us/step - loss: 0.6209 - acc: 0.7676 - val_loss: 0.9241 - val_acc: 0.6630
Epoch 14/50
1652/1652 [==============================] - 0s 126us/step - loss: 0.5863 - acc: 0.7900 - val_loss: 1.1052 - val_acc: 0.5870
Epoch 15/50
1652/1652 [==============================]

1652/1652 [==============================] - 0s 142us/step - loss: 0.4141 - acc: 0.8438 - val_loss: 1.0642 - val_acc: 0.6304
Epoch 25/50
1652/1652 [==============================] - 0s 131us/step - loss: 0.3989 - acc: 0.8553 - val_loss: 0.9977 - val_acc: 0.6522
Epoch 26/50
1652/1652 [==============================] - 0s 130us/step - loss: 0.3607 - acc: 0.8723 - val_loss: 2.1608 - val_acc: 0.3967
Epoch 27/50
1652/1652 [==============================] - 0s 135us/step - loss: 0.3335 - acc: 0.8826 - val_loss: 1.0589 - val_acc: 0.6630
Epoch 28/50
1652/1652 [==============================] - 0s 135us/step - loss: 0.3443 - acc: 0.8729 - val_loss: 1.0047 - val_acc: 0.6957
Epoch 29/50
1652/1652 [==============================] - 0s 131us/step - loss: 0.3294 - acc: 0.8826 - val_loss: 1.3823 - val_acc: 0.5870
Epoch 30/50
1652/1652 [==============================] - 0s 130us/step - loss: 0.3162 - acc: 0.8886 - val_loss: 1.0336 - val_acc: 0.6902
Epoch 31/50
1652/1652 [==============================

Epoch 40/50
1652/1652 [==============================] - 0s 135us/step - loss: 0.2121 - acc: 0.9201 - val_loss: 1.1244 - val_acc: 0.6359
Epoch 41/50
1652/1652 [==============================] - 0s 136us/step - loss: 0.2247 - acc: 0.9231 - val_loss: 1.3144 - val_acc: 0.5978
Epoch 42/50
1652/1652 [==============================] - 0s 138us/step - loss: 0.1915 - acc: 0.9364 - val_loss: 1.5045 - val_acc: 0.6141
Epoch 43/50
1652/1652 [==============================] - 0s 138us/step - loss: 0.1976 - acc: 0.9346 - val_loss: 1.4068 - val_acc: 0.6196
Epoch 44/50
1652/1652 [==============================] - 0s 146us/step - loss: 0.1924 - acc: 0.9292 - val_loss: 1.2595 - val_acc: 0.6196
Epoch 45/50
1652/1652 [==============================] - 0s 128us/step - loss: 0.2002 - acc: 0.9334 - val_loss: 1.2604 - val_acc: 0.7011
Epoch 46/50
1652/1652 [==============================] - 0s 134us/step - loss: 0.2065 - acc: 0.9255 - val_loss: 1.3728 - val_acc: 0.6522
Epoch 47/50
1652/1652 [==================

1652/1652 [==============================] - 0s 95us/step - loss: 0.8134 - acc: 0.7137 - val_loss: 0.8310 - val_acc: 0.6957
Epoch 7/50
1652/1652 [==============================] - 0s 92us/step - loss: 0.7811 - acc: 0.7173 - val_loss: 0.8665 - val_acc: 0.7011
Epoch 8/50
1652/1652 [==============================] - 0s 93us/step - loss: 0.7208 - acc: 0.7494 - val_loss: 0.8975 - val_acc: 0.6467
Epoch 9/50
1652/1652 [==============================] - 0s 90us/step - loss: 0.6987 - acc: 0.7554 - val_loss: 0.8721 - val_acc: 0.6793
Epoch 10/50
1652/1652 [==============================] - 0s 94us/step - loss: 0.6563 - acc: 0.7748 - val_loss: 0.7857 - val_acc: 0.7120
Epoch 11/50
1652/1652 [==============================] - 0s 93us/step - loss: 0.6263 - acc: 0.7809 - val_loss: 0.8242 - val_acc: 0.7011
Epoch 12/50
1652/1652 [==============================] - 0s 91us/step - loss: 0.5999 - acc: 0.7839 - val_loss: 0.9655 - val_acc: 0.6522
Epoch 13/50
1652/1652 [==============================] - 0s 99u

1652/1652 [==============================] - 0s 94us/step - loss: 0.3330 - acc: 0.8862 - val_loss: 0.9093 - val_acc: 0.6630
Epoch 25/50
1652/1652 [==============================] - 0s 98us/step - loss: 0.3286 - acc: 0.8801 - val_loss: 0.9775 - val_acc: 0.6739
Epoch 26/50
1652/1652 [==============================] - 0s 92us/step - loss: 0.3133 - acc: 0.8923 - val_loss: 1.0101 - val_acc: 0.6793
Epoch 27/50
1652/1652 [==============================] - 0s 94us/step - loss: 0.2949 - acc: 0.9001 - val_loss: 1.0163 - val_acc: 0.6413
Epoch 28/50
1652/1652 [==============================] - 0s 94us/step - loss: 0.2659 - acc: 0.9146 - val_loss: 1.2925 - val_acc: 0.6087
Epoch 29/50
1652/1652 [==============================] - 0s 90us/step - loss: 0.2650 - acc: 0.9068 - val_loss: 1.1428 - val_acc: 0.6848
Epoch 30/50
1652/1652 [==============================] - 0s 97us/step - loss: 0.2370 - acc: 0.9177 - val_loss: 1.1680 - val_acc: 0.7011
Epoch 31/50
1280/1652 [======================>.......] - ETA